## get_user_recommendation

In [1]:
import utils.load_data as load_data
import utils.Dataload as Dataload
from pathlib import Path
from surprise.model_selection import train_test_split
from surprise import SVD, NMF
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.algo_base import AlgoBase
from surprise.dump import dump, load

In [2]:
data_raw = data_loader.get_data('ml-100k')
data_dir = Path('../data/movielens/ml-latest-small/ratings.csv')
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data_raw = data_loader.get_data('ratings.csv', False)

In [3]:
def train_and_evalute_model_pipeline(model_class: AlgoBase, dataset: str = 'ml-100k', 
                                     from_surprise: bool = True, 
                                     test_size: float = 0.2,
                                     model_kwargs: dict = {}) -> (AlgoBase, dict):
    data = data_loader.get_data(dataset, from_surprise)
    train_set, test_set = train_test_split(data, test_size, random_state=42)
    model = Dataload.get_trained_model(model_class, train_set, model_kwargs)
    metrics_dict = Dataload.evaluate_model(model, test_set)
    return model, metrics_dict

In [10]:
my_model, metrics_dict = train_and_evalute_model_pipeline(KNNBasic)
metrics_dict

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [9]:
def get_user_recommendation(model: AlgoBase, user_id: int, k: int) -> pd.DataFrame:
    
    try:
        preds, model = load(Path(model_dir, str(model)[8:-2]))
    except:
        data = data_loader.get_data('ml-100k')
        trainset = data.build_full_trainset()
        testset = trainset.build_anti_testset()
        trained_model = model_loader.get_trained_model(model, trainset)
        preds = trained_model.test(testset)
        dump(Path(model_dir, str(model)[8:-2]), preds, trained_model)
    top_n = predictions.get_top_n(preds, n=k)
    movies = data_loader.get_item_data('movies.csv', data_dir)

    
    return predictions.get_item_details(top_n[user_id], movies, 'movieId', ['movieId', 'title', 'genres'])

recommendations = get_user_recommendation(SVD, '196', 10)
recommendations.reset_index(drop=True, inplace=True)
display(recommendations)